# Advanced: Iterating over genomes with PatMatch

The [previous notebook, 'Advanced: Sending PatMatch output directly to Python'](Sending%20PatMatch%20output%20directly%20to%20Python.ipynb), 
[Advanced: Sending PatMatch output directly to Python](notebooks/Sending%20PatMatch%20output%20directly%20to%20Python.ipynb)
covered leveraging the the Jupyter environment to skip over needing to save a file to actually pass results from shell scripts into Python. This notebook will demonstrate using one of those approaches to iterate over several genomes.

## Preparing

Similar to the previous notebook, in order to insure everything is all set, act as if this is a new session in this Jupyter environment, and run the next cell so that you can start stepping through the preparation steps by first getting a sequence file. Plus, you'll get the file for script to convert it to dataframe and import its the main function.

Repeating these steps if you had already done so this session will cause no harm, and so go ahead and run this cell.

In [1]:
!curl -O https://downloads.yeastgenome.org/sequence/S288C_reference/chromosomes/fasta/chrmt.fsa
!curl -O https://raw.githubusercontent.com/fomightez/sequencework/master/patmatch-utilities/patmatch_results_to_df.py
from patmatch_results_to_df import patmatch_results_to_df

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 87344  100 87344    0     0   236k      0 --:--:-- --:--:-- --:--:--  236k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18722  100 18722    0     0    99k      0 --:--:-- --:--:-- --:--:--   99k


Additionally, the sequences of two other mitochondrial genomes will be retrieved.

Reference for the additional sequence data:  
- [Contrasting evolutionary genome dynamics between domesticated and wild yeasts.
Yue JX, Li J, Aigrain L, Hallin J, Persson K, Oliver K, Bergström A, Coupland P, Warringer J, Lagomarsino MC, Fischer G, Durbin R, Liti G. Nat Genet. 2017 Jun;49(6):913-924. doi: 10.1038/ng.3847. Epub 2017 Apr 17. PMID: 28416820](https://www.ncbi.nlm.nih.gov/pubmed/28416820)

In [2]:
# Prepare for getting PacBio (Yue et al 2017 sequences)
#make a list of the strain designations
yue_et_al_strains = ["CBS432","N44"]
# Get & unpack the genome sequences from strains 
import os
genomes = []
expected_resulting_file = "N44_mito.genome.fa"
if not os.path.isfile(expected_resulting_file):
    for s in yue_et_al_strains:
        !curl -OL http://yjx1217.github.io/Yeast_PacBio_2016/data/Mitochondrial_Genome/{s}.mt.genome.fa.gz
        !gunzip -f {s}.mt.genome.fa.gz
        # rename the files to follow the convention used for SGD reference
        !mv {s}.mt.genome.fa {s}_mito.genome.fsa
        genomes.append(s+"_mito.genome.fsa")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0    601      0 --:--:-- --:--:-- --:--:--   599
100 18692  100 18692    0     0  42385      0 --:--:-- --:--:-- --:--:-- 42385
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   178  100   178    0     0   2170      0 --:--:-- --:--:-- --:--:--  2170
100 18399  100 18399    0     0  80697      0 --:--:-- --:--:-- --:--:-- 80697


Add identifiers to each description line so results for each strain clear later. The reference from the Saccharomyces Genome database will be tagged 'SGD_REFmito'.

In [10]:
%%capture
import sys
import os
# add identifiers to each description line so results for each strain clear later
def add_strain_id_to_description_line(file,strain_id):
    '''
    Takes a file and edits every description line to add 
    strain_id after the caret.
    
    Saves the fixed file
    '''
    import sys
    output_file_name = "temp.txt"
    # prepare output file for saving so it will be open and ready
    with open(output_file_name, 'w') as output_file:

        # read in the input file
        with open(file, 'r') as input_handler:
            # prepare to give feeback later or allow skipping to certain start
            lines_processed = 0

            for line in input_handler:
                lines_processed += 1
                if line.startswith(">"):
                    rest_o_line = line.split(">")
                    new_line = ">"+strain_id +"\n"
                else:
                    new_line = line
                
                # Send text to output
                output_file.write(new_line)

    
    # replace the original file with edited
    !mv temp.txt {file}
    # Feedback
    sys.stderr.write("\n{} has had identifiers added.".format(file))

files_tagged = 0
for g in genomes:
    add_strain_id_to_description_line(g, g.split('.genome.fsa')[0])
    files_tagged += 1
# Feedback
sys.stderr.write("\n{} sets of strain identifiers added.".format(files_tagged))
# Edit the description line for the SGD reference to be clear
!sed -i '1s/.*/>SGD_REFmito/' chrmt.fsa

Make a list of the genomes based on the name.

In [7]:
fn_pat_to_check = ".fsa"
genomes = []
import os
import fnmatch
for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*'+fn_pat_to_check):
        genomes.append(file)
genomes

['chrmt.fsa', 'N44_mito.genome.fsa', 'CBS432_mito.genome.fsa']

## Running Patmatch and passing the results to Python without creating an output file intermediate

#### Option 1

First we'll do one of the several methods I have found to do this and show how to go to the next step entirely. This first example uses an approach illustrated [here](https://stackoverflow.com/a/42703609/8508004).  The result that is returned is of the type `IPython.utils.text.SList`, which offers some handy utility attributes associated with it as detailed [here](http://ipython.readthedocs.io/en/stable/api/generated/IPython.utils.text.html#IPython.utils.text.SList).

In [ ]:
output = !perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared 

That performed the matching step without generating an intermediate file. Next, run the next cell to combine this with use of main function of `patmatch_results_to_df.py` as illustrated in the [previous notebook, 'PatMatch with more Python'](PatMatch with more Python.ipynb).

In [ ]:
#patmatch_results_to_df.py with the `.n` attribute of IPython.utils.text.SList
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(output.n, pattern=my_pattern, name="promoter")

This option has the distinction that all the steps in it can be combined in one cell of the notebook. This is because the `!<command>` approach makes a temporary shell outside of the notebook environment to which it sends the command after the exclamation site. After that command is processed and any communication handled, that subshell where it ran is discarded. Given the transient nature of this environment you'll find `!cd` never seems to work as they discuss [here](http://ipython.readthedocs.io/en/stable/interactive/magics.html), search `!cd` to find the pertinent section where they also show you the line magics solution. ([Here](https://jakevdp.github.io/PythonDataScienceHandbook/01.05-ipython-and-shell-commands.html) is another good resource in this regard.)

This option can be worked into a utility function that includes the preparation step. This is very handy if trying a lot of patterns and/or sequences because could easily be called in a loop and supplied with different parameters. 

In [ ]:
def patmatch_query(pattern, seq_file, name):
    '''
    function that will work in Jupyter lab notebooks to run Patmatch query.
    
    Requires pattern and sequence file. Plus a name to use to refer to 
    instances of pattern in output.
    
    Returns dataframe of results.
    '''
    from patmatch_results_to_df import patmatch_results_to_df

    !perl ../patmatch_1.2/unjustify_fasta.pl {seq_file}
    output = !perl ../patmatch_1.2/patmatch.pl -c {pattern} {seq_file+".prepared"}
    #!rm {seq_file+".prepared"} # OPTIONAL deleting of prepared file.
    df = patmatch_results_to_df(output.n, pattern=pattern, name=name)
    return df

(I commented out deleting the preparation file in the function in the above cell because I want to use it later on in this notebook, but it would be useful to include normally to not generate a lot of uncessary files.)

In [ ]:
df = patmatch_query(my_pattern, "chrmt.fsa", "promoter")

----

The additional options for passing the results demonstrated below instead rely on cell magics, and so the output needs to be captured from a cell before the next steps can be undertaken in an additional cell. While not a big deal that extra cells are involved, I find the `!<command>` approach can be nice for streamlining things when making mini-pipelines/workflows using the Jupyter environment as a 'glue' to merge Python and command line use. 

The additional options for passing the results will be stepped through in a manner similar to 'Option 1' using different approaches for step 1 each time and step #2 varied to handle as necessary.

#### Option 2

In this option, [`%%capture` cell magics](http://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-capture) is used, and then using the attributes of the `utils.cpature` object you can easily get the stdout and/or stderr as a string, see [here]( http://ipython.readthedocs.io/en/stable/api/generated/IPython.utils.capture.html).

In [ ]:
%%capture out
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared

In [ ]:
#patmatch_results_to_df.py with the `.stdout` attribute of `utils.cpature`
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(out.stdout, pattern=my_pattern, name="promoter")

#### Option 3

In this option, a varation of `%%bash` cell magic is used to send the output to a variable as illustrated [here](https://stackoverflow.com/a/24776049/8508004).

In [ ]:
%%bash --out output
perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared

In [ ]:
#patmatch_results_to_df.py by passing the stdout to a varible using %%bash cell magic
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(output, pattern=my_pattern, name="promoter")

#### Option 4 (<font color='red'>** NEVERMIND, THIS DOESN'T SEEM TO WORK.**</font>** NEVERMIND, THIS DOESN'T SEEM TO WORK.**)

In this option, IPython's output caching system as summarized [here](https://stackoverflow.com/a/27952661/8508004) is used. Step#1 requires nothing different than the normal command that sends the output to the output stream of the cell. The trick comes in step#2 where that output is used by pointing to it with an underscore. <font color='red'>** DESPITE THE FACT I HAD NOTED THIS APPROACH WORKED ONCE IN THE PAST WITH A RESULT FROM THE SHELL, IT DOESN'T SEEM TO WORK AND MAYBE I WAS FOOLED BY SOMETHING IN THE NAMESPACE?!?**</font>

In [ ]:
!perl ../patmatch_1.2/patmatch.pl -c "DDWDWTAWAAGTARTADDDD" chrmt.fsa.prepared 

In [ ]:
#patmatch_results_to_df.py by reference the output of the previous cell using IPython's 'output caching system'
my_pattern= "DDWDWTAWAAGTARTADDDD"
df = patmatch_results_to_df(_, pattern=my_pattern, name="promoter")